In [1]:
import pandas as pd
import numpy as np
from helpers import tts_scale
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [3]:
df[df['Credit Risk Code'].isna()]

,PK,Date,Qdate,Balance,Days Past Due,Past_Due30_59,Past_Due60_89,Past_Due90+,Credit Risk Code,Adversity,...,DEVELOPING_ASIA_REAL_GDP_GROWTH,DEVELOPING_ASIA_INFLATION,DEV_ASIA_BILATERAL_DOLLAR_EXCHANGE_RATE,JAPAN_REAL_GDP_GROWTH,JAPAN_INFLATION,JAPAN_BILATERAL_DOLLAR_EXCHANGE_RATE,UK_REAL_GDP_GROWTH,UK_INFLATION,UK_BILATERAL_DOLLAR_EXCHANGE_RATE,Default
237185,4598,2017-08-31,Q3 2017,29574.34,0,0,0,0,NaN,Historical,...,7.1,2.2,93.7,2.7,0.5,112.6,1.4,2.3,1.340,0.0
240133,4598,2017-09-30,Q3 2017,29115.63,0,0,0,0,NaN,Historical,...,7.1,2.2,93.7,2.7,0.5,112.6,1.4,2.3,1.340,0.0
276814,4598,2017-10-31,Q4 2017,28414.96,0,0,0,0,NaN,Historical,...,6.0,2.9,91.1,1.2,1.5,112.7,1.6,2.9,1.353,0.0
277205,6554,2017-10-31,Q4 2017,238540.42,0,0,0,0,NaN,Historical,...,6.0,2.9,91.1,1.2,1.5,112.7,1.6,2.9,1.353,0.0


In [4]:
df = pd.read_csv(r'../data/final.csv', index_col=0)
df = df.dropna()
df['Credit Risk Code'] = df['Credit Risk Code'].astype(int).astype(str)
df.head()

,PK,Date,Qdate,Balance,Days Past Due,Past_Due30_59,Past_Due60_89,Past_Due90+,Credit Risk Code,Adversity,...,DEVELOPING_ASIA_REAL_GDP_GROWTH,DEVELOPING_ASIA_INFLATION,DEV_ASIA_BILATERAL_DOLLAR_EXCHANGE_RATE,JAPAN_REAL_GDP_GROWTH,JAPAN_INFLATION,JAPAN_BILATERAL_DOLLAR_EXCHANGE_RATE,UK_REAL_GDP_GROWTH,UK_INFLATION,UK_BILATERAL_DOLLAR_EXCHANGE_RATE,Default
0,0,2015-01-31,Q1 2015,7685.03,15,10,5,1,5,Historical,...,5.7,0.9,88.1,5.5,0.5,120.0,2.1,-1.1,1.485,1.0
1,1,2015-01-31,Q1 2015,181479.00,0,0,0,0,3,Historical,...,5.7,0.9,88.1,5.5,0.5,120.0,2.1,-1.1,1.485,0.0
2,2,2015-01-31,Q1 2015,67382.89,0,0,0,0,3,Historical,...,5.7,0.9,88.1,5.5,0.5,120.0,2.1,-1.1,1.485,0.0
3,3,2015-01-31,Q1 2015,115580.00,0,0,0,0,1,Historical,...,5.7,0.9,88.1,5.5,0.5,120.0,2.1,-1.1,1.485,0.0
4,4,2015-01-31,Q1 2015,111445.00,0,0,0,0,1,Historical,...,5.7,0.9,88.1,5.5,0.5,120.0,2.1,-1.1,1.485,0.0


# Create 3 groups of Credit Risk Code
## Reliable ==> 1,2,3
## Semi-Reliable ==> 4,5,6,7
## Unreliable ==> 8,9,10

In [6]:
df.loc[df['Credit Risk Code'].isin(['1','2','3']),'Credit Bagged'] = '0'
df.loc[df['Credit Risk Code'].isin(['4','5','6','7']),'Credit Bagged'] = '1'
df.loc[df['Credit Risk Code'].isin(['8','9','10']),'Credit Bagged'] = '2'

In [24]:
df.groupby(['Credit Bagged'])['Credit Bagged'].agg({'Credit Bagged':'count'})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,Credit Bagged
Credit Bagged,
0,178614
1,114701
2,2659


In [23]:
df.groupby(['Credit Bagged', 'Credit Risk Code']).agg({'Credit Risk Code':'count'})

Credit Risk Code
Credit Bagged Credit Risk Code                  
0             1                             9791
              2                              510
              3                           168313
1             4                            84239
              5                            13536
              6                             1413
              7                            15513
2             10                             443
              8                             1954
              9                              262

In [8]:
df.columns

Index(['PK', 'Date', 'Qdate', 'Balance', 'Days Past Due', 'Past_Due30_59',
       'Past_Due60_89', 'Past_Due90+', 'Credit Risk Code', 'Adversity',
       'QTR_CLOSE', 'REAL_GDP_GROWTH', 'NOMINAL_GDP_GROWTH',
       'REAL_DISPOSABLE_INCOME_GROWTH', 'NOMINAL_DISPOSABLE_INCOME_GROWTH',
       'UNEMPLOYMENT_RATE', 'CPI_INFLATION_RATE', 'THREE_MONTH_TREASURY_RATE',
       'FIVE_YEAR_TREASURY_YIELD', 'TEN_YEAR_TREASURY_YIELD',
       'BBB_CORPORATE_YEILD', 'MORTGAGE_RATE', 'PRIME_RATE',
       'DOWJONES_TOTAL_STOCK_MARKET_INDEX', 'HOUSE_PRICE_INDEX',
       'COMERCIAL_REAL_ESTATE_PRICE_INDEX', 'MARKET_VOLATILITY_INDEX',
       'EURO_AREA_REAL_GDP_GROWTH', 'EURO_AREA_INFLATION',
       'EURO_AREA_BILATERAL_DOLLAR_EXCHANGE_RATE',
       'DEVELOPING_ASIA_REAL_GDP_GROWTH', 'DEVELOPING_ASIA_INFLATION',
       'DEV_ASIA_BILATERAL_DOLLAR_EXCHANGE_RATE', 'JAPAN_REAL_GDP_GROWTH',
       'JAPAN_INFLATION', 'JAPAN_BILATERAL_DOLLAR_EXCHANGE_RATE',
       'UK_REAL_GDP_GROWTH', 'UK_INFLATION',
       'UK_

In [10]:
#X = df[['Days Past Due', 'Balance', 'Past_Due30_59', 'Past_Due60_89', 'Past_Due90+', 'Credit Bagged']]
X = df.drop(columns=['PK', 'Date', 'Qdate','Credit Risk Code','Adversity','QTR_CLOSE','Default'])

In [11]:
len(X)

295974

In [12]:
x_train, x_test = train_test_split(X, random_state=42, test_size = 0.3)
train_bag = x_train['Credit Bagged']
test_bag = x_test['Credit Bagged']
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train.drop(columns=['Credit Bagged']))
x_test = scaler.transform(x_test.drop(columns=['Credit Bagged']))

C:\Users\smr6743\AppData\Roaming\Python\Python37\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\smr6743\AppData\Roaming\Python\Python37\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [13]:
clf = KMeans(n_clusters=3, 
             random_state=42).fit(x_train)

In [14]:
bindict = {'0':'Reliable', '1':'Semi-Reliable','2':'Unreliable'}
rownames = []
for val in train_bag.values:
    rownames.append(bindict[val])
    
colnames = []
for val in clf.labels_:
    colnames.append(bindict[str(val)])

In [16]:
print(accuracy_score(train_bag, clf.labels_.astype(str)))
print(classification_report(train_bag, clf.labels_.astype(str)))
crosstab_train = pd.crosstab(train_bag, clf.labels_)
crosstab_train.columns = ['0','1','2']
crosstab_train.rename(columns=bindict, index = bindict, inplace=True)
crosstab_train

0.3760383432843745
              precision    recall  f1-score   support

           0       0.57      0.17      0.26    124910
           1       0.38      0.71      0.49     80382
           2       0.01      0.07      0.01      1889

   micro avg       0.38      0.38      0.38    207181
   macro avg       0.32      0.31      0.25    207181
weighted avg       0.49      0.38      0.35    207181



,Reliable,Semi-Reliable,Unreliable
Credit Bagged,,,
Reliable,21047,92785,11078
Semi-Reliable,15457,56730,8195
Unreliable,194,1564,131


In [17]:
clf.inertia_

3479625.0149924755

In [18]:
predictions = clf.predict(x_test)

In [19]:
print(accuracy_score(test_bag, predictions.astype(str)))
print(classification_report(test_bag, predictions.astype(str)))
crosstab_test = pd.crosstab(test_bag, predictions.astype(str))
crosstab_test.columns = ['0','1','2']
crosstab_test.rename(columns=bindict, index = bindict, inplace=True)
crosstab_test

0.3752097575259311
              precision    recall  f1-score   support

           0       0.58      0.17      0.26     53704
           1       0.37      0.70      0.49     34319
           2       0.01      0.06      0.01       770

   micro avg       0.38      0.38      0.38     88793
   macro avg       0.32      0.31      0.25     88793
weighted avg       0.49      0.38      0.35     88793



,Reliable,Semi-Reliable,Unreliable
Credit Bagged,,,
Reliable,9111,39870,4723
Semi-Reliable,6613,24158,3548
Unreliable,84,639,47
